In [124]:
import numpy as np
import cv2
from cv2 import dnn
import os
import PIL
from PIL import Image, ImageEnhance

In [125]:
#--------Model file paths--------#
proto_file = 'Model\colorization_deploy_v2.prototxt'
model_file = 'Model\colorization_release_v2.caffemodel'
hull_pts = 'Model\pts_in_hull.npy'

In [126]:

#--------Save locations--------#
fileName= '\img1.jpg'
saveName= '\output.jpg'
un_saturated_path = 'images\gray'+fileName
saturated_path = 'images\saturated'+fileName
org_img_path = 'images\colored'+fileName
img_path = 'images\saturated'+fileName
result_save_location = 'images\output'
#--------Reading the model params--------#
net = dnn.readNetFromCaffe(proto_file,model_file)
kernel = np.load(hull_pts)

#--------Turn the image into gray--------#
gray = Image.open(org_img_path).convert('L')
gray.save(un_saturated_path)
#-----Reading and preprocessing image--------#
im = Image.open(r'images/gray/img1.jpg')
im3 = ImageEnhance.Color(im)
im3 = im3.enhance(2.5)
im3 = im3.save(saturated_path,"JPEG")
img = cv2.imread(img_path)

colored_image = cv2.imread(org_img_path)

original = cv2.imread(un_saturated_path)
#original = cv2.imread(saturated_path)

scaled = img.astype("float32") / 255.0
lab_img = cv2.cvtColor(scaled, cv2.COLOR_BGR2LAB)
#-----------------------------------#---------------------#
 
# add the cluster centers as 1x1 convolutions to the model
class8 = net.getLayerId("class8_ab")
conv8 = net.getLayerId("conv8_313_rh")
pts = kernel.transpose().reshape(2, 313, 1, 1)
net.getLayer(class8).blobs = [pts.astype("float32")]
net.getLayer(conv8).blobs = [np.full([1, 313], 2.606, dtype="float32")]
#-----------------------------------#---------------------#

In [127]:
# add the cluster centers as 1x1 convolutions to the model
class8 = net.getLayerId("class8_ab")
conv8 = net.getLayerId("conv8_313_rh")
pts = kernel.transpose().reshape(2, 313, 1, 1)
net.getLayer(class8).blobs = [pts.astype("float32")]
net.getLayer(conv8).blobs = [np.full([1, 313], 2.606, dtype="float32")]

In [128]:
# we'll resize the image for the network
resized = cv2.resize(lab_img, (224, 224))
# split the L channel
L = cv2.split(resized)[0]
# mean subtraction
L -= 50

In [129]:
# predicting the ab channels from the input L channel
 
net.setInput(cv2.dnn.blobFromImage(L))
ab_channel = net.forward()[0, :, :, :].transpose((1, 2, 0))
# resize the predicted 'ab' volume to the same dimensions as our
# input image
ab_channel = cv2.resize(ab_channel, (img.shape[1], img.shape[0]))
 
 
# Take the L channel from the image
L = cv2.split(lab_img)[0]
# Join the L channel with predicted ab channel
colorized = np.concatenate((L[:, :, np.newaxis], ab_channel), axis=2)
 
# Then convert the image from Lab to BGR
colorized = cv2.cvtColor(colorized, cv2.COLOR_LAB2BGR)
colorized = np.clip(colorized, 0, 1)
 
# change the image to 0-255 range and convert it from float32 to int
colorized = (255 * colorized).astype("uint8")

#Use this if the resolution of the images become too large
colored_image_viewing = cv2.resize(colored_image,(960,640))
colorized_viewing = cv2.resize(colorized,(960,640))
result = cv2.hconcat([colored_image_viewing,colorized_viewing])


#result = cv2.hconcat([colored_image,colorized])

#Checks the difference of the rgb values, the closer to 0 the more similar it is
difference = cv2.subtract(original, colorized)
b, g, r = cv2.split(difference)
difR = cv2.countNonZero(r)
difG = cv2.countNonZero(g)
difB = cv2.countNonZero(b)
print("R:",difR," G:", difG," B:", difB)



cv2.imwrite(os.path.join(result_save_location, 'output.jpg'),colorized)

cv2.imshow("Grayscale -> Colour", result)
 
cv2.waitKey(0)

R: 539267  G: 598991  B: 1024935


-1